Курсовой проект для курса "Python для Data Science"

Материалы к проекту (файлы):
train.csv
test.csv

Задание:
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

Целевая переменная:
Price

Основная метрика:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Вспомогательная метрика:
MSE - средняя квадратичная ошибка (sklearn.metrics.mean_squared_error)

Сдача проекта:
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту")
ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook 
с расширением ipynb).
2. Приложить файл с названием по образцу PNagornyi_predictions.csv
с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price).

Сроки сдачи:
Сдать проект за 72 часа после Урока 10 ("Вебинар. Консультация по итоговому проекту").

Примечание:
Все файлы csv должны содержать названия полей (header - то есть "шапку"),
разделитель - запятая. В файлах не должны содержаться индексы из датафрейма.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR, Ridge
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import r2_score as r2, mean_squared_error as MSE

In [2]:
db = pd.read_csv('train.csv')
data = db.copy()
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [4]:
#т.к. очень много пропущенных данных, иключим из анализа признак Healthcare_1
data = data.drop('Healthcare_1', axis=1)

In [5]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


Исследуем признак `Rooms` - количество комнат.

In [6]:
data.loc[data.Rooms>5]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,5,15,B,317265.323792
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,B,B,1,264,0,0,1,B,78364.616704
2170,14003,99,6.0,59.414334,38.702244,6.0,7,9.0,1969,0.033494,B,B,66,10573,1,3,8,B,229661.964416
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994,0.161532,B,B,25,5648,1,2,4,B,172329.270863


In [7]:
# гипотеза №1, приведем кол-во комнат > 5 к кол-ву комнат в зависимости от Square
data.loc[data.Rooms==10, 'Rooms']=2
data.loc[data.Rooms==6, 'Rooms']=2
data.loc[1454, 'Rooms']=1

In [8]:
data.loc[data.Rooms<1]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,0,0,B,268394.744389
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,0,1,B,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1,0,B,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,0,1,B,219281.918007


In [9]:
# гипотеза №2, считаем квартиры с Rooms=0, квартирами Rooms=1
data.loc[data.Rooms<1, 'Rooms']=1

Считаем, что признак `Rooms` мы исследовали.

Исследуем признакы `KitchenSquare`- площадь кухни, `LifeSquare` - жилая площадь квартиры и `Square` - площадь квартиры.

In [10]:
data.loc[data.KitchenSquare<2]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,1,3,B,220925.908524
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,0,2,B,221244.156664
8,5172,1,2.0,64.511437,NaN,1.0,9,17.0,1977,0.007122,B,B,1,264,0,0,1,B,229102.795999
12,6452,13,2.0,54.522805,31.759753,1.0,8,12.0,1999,0.090799,B,B,74,19083,2,5,15,B,229220.372609
14,10953,27,1.0,53.769097,52.408027,1.0,5,4.0,1977,0.072158,B,B,2,629,1,0,0,A,140647.565937
21,11935,27,2.0,64.711835,NaN,1.0,15,1.0,1977,0.211401,B,B,9,1892,0,0,1,B,127200.026511
23,6641,54,3.0,118.907612,NaN,0.0,2,0.0,1977,0.006076,B,B,30,5285,0,6,6,B,571069.052600
25,5073,74,2.0,82.667915,NaN,1.0,16,17.0,1977,0.075779,B,B,6,1437,3,0,2,B,244145.827069
26,4378,27,3.0,106.958871,0.641822,0.0,17,0.0,2018,0.072158,B,B,2,629,1,0,0,A,337299.867936
27,7550,23,2.0,64.556209,NaN,1.0,8,17.0,1977,0.075779,B,B,6,1437,3,0,2,B,173910.923723


In [11]:
mean_kitchen_sq=data.groupby(['Rooms'],as_index=False)[['KitchenSquare']].mean().rename(columns={'KitchenSquare':'mean_KitchenSquare'})
mean_kitchen_sq.head()

,Rooms,mean_KitchenSquare
0,1.0,6.514270
1,2.0,5.638681
2,3.0,6.801790
3,4.0,8.506667
4,5.0,9.222222


In [12]:
# гипотеза №3
# Т.к. нет четкой зависимости средней площади кухни от количества комнат
# Площадь кухни меньше 2 заполняем средней площадью кухни.
data.loc[data.KitchenSquare<2, 'KitchenSquare'] = data.KitchenSquare.mean()
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.887500,56.315775,37.199645,8.007781,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.811486,21.058732,86.241209,28.339594,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,1.136859,0.370619,2.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,6.273300,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.273300,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,5.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


In [13]:
data.loc[data.KitchenSquare>50]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
227,16395,2,3.0,79.722243,44.731219,72.0,12,16.0,1987,0.130618,B,B,39,10418,9,1,9,B,370148.625285
1064,14656,62,1.0,47.100719,46.447960,2014.0,4,1.0,2014,0.072158,B,B,2,629,1,0,0,A,108337.484207
1369,2371,27,2.0,68.841073,64.234956,66.0,4,2.0,2014,0.017647,B,B,2,469,0,0,0,B,189244.249909
1455,12507,54,2.0,79.810535,79.578961,78.0,10,15.0,2014,0.006076,B,B,30,5285,0,6,6,B,438708.707579
1860,4265,161,2.0,53.216778,32.644859,53.0,7,17.0,1994,0.000699,B,B,14,3369,24,0,3,B,261125.669724
2916,12390,72,3.0,97.490674,99.323558,96.0,22,25.0,2019,0.210473,B,B,11,2398,2,3,0,B,445074.956552
2969,7441,62,3.0,114.734473,112.589083,112.0,3,3.0,2015,0.072158,B,B,2,629,1,0,0,A,315245.521059
4079,6508,23,2.0,67.146049,33.959154,63.0,5,17.0,2019,0.034656,B,B,0,168,0,0,0,B,193130.585871
4110,299,27,2.0,66.787523,64.616662,60.0,14,20.0,2015,0.017647,B,B,2,469,0,0,0,B,179466.094235
4651,12552,58,3.0,116.405693,113.109653,112.0,3,3.0,2016,0.437885,B,B,23,5735,3,0,5,B,296165.936689


In [14]:
# гипотеза №4
# посчитаем площадь кухни больше 50 - аномальной и заполним средним значением
data.loc[data.KitchenSquare>50, 'KitchenSquare'] = data.KitchenSquare.mean()

In [15]:
# получим среднее значение по Square, в которых данные по LifeSquare заполнены
data.loc[data.LifeSquare>0, 'Square'].mean()

# получим среднее значение по KitchenSquare, в которых данные по LifeSquare заполнены
data.loc[data.LifeSquare>0, 'KitchenSquare'].mean()

# получим среднее значение по LifeSquare, в которых данные по LifeSquare заполнены
data.loc[data.LifeSquare>0, 'LifeSquare'].mean()

# гипотеза №5 
# считаем, что среднее значение прочей площади(не KitchenSquare и LifeSquare)
Square_other_mean = data.loc[data.LifeSquare>0, 'Square'].mean() - data.loc[data.LifeSquare>0, 'KitchenSquare'].mean() - data.loc[data.LifeSquare>0, 'LifeSquare'].mean()
Square_other_mean

10.290016701809868

In [16]:
# посмотрим на Square < 20
data.loc[(data.Square<20)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
212,1748,88,2.0,5.497061,67.628717,6.2733,24,22.0,1977,0.127376,B,B,43,8429,3,3,9,B,412511.088764
1316,11526,27,1.0,4.633498,1.969969,6.2733,18,1.0,1977,0.011654,B,B,4,915,0,0,0,B,107604.269441
1608,10202,6,1.0,2.596351,4.604943,6.2733,3,25.0,2014,0.243205,B,B,5,1564,0,0,0,B,137597.601458
1891,13685,0,1.0,17.498542,13.446544,6.2733,7,5.0,1961,0.319809,B,B,25,4756,16,5,8,B,88871.750230
3166,15127,150,1.0,17.046188,20.036232,8.0000,7,12.0,1982,0.300323,B,B,52,10311,6,1,9,B,74222.110465
3280,10527,27,1.0,4.380726,40.805837,6.2733,10,17.0,2013,0.211401,B,B,9,1892,0,0,1,B,97560.720383
3413,9487,5,1.0,5.129222,5.549458,6.2733,1,1.0,1977,0.150818,B,B,16,3433,4,4,5,B,369472.403061
3794,523,101,1.0,18.509177,15.170851,3.0000,5,5.0,1970,0.225825,A,B,41,6558,7,3,2,B,115845.861527
4739,12676,81,3.0,13.784865,15.988889,7.0000,4,5.0,1960,0.319809,B,B,25,4756,16,5,8,B,78388.806186
4853,3224,27,1.0,2.377248,0.873147,6.2733,1,0.0,1977,0.017647,B,B,2,469,0,0,0,B,126596.941798


In [17]:
# гипотеза №6
# скорректируем данные [Square < 20] = LifeSquare + KitchenSquare + Square_other_mean
# для этого исключим из выборки пустые значения LifeSquare
data.loc[(data.Square<20) & (data.LifeSquare>0), 'Square'] = data['LifeSquare'] + data['KitchenSquare'] + Square_other_mean

In [18]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.887500,56.364381,37.199645,7.486097,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.811486,20.996538,86.241209,2.519404,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,17.414181,0.370619,2.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.782323,22.769832,6.273300,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.535436,32.781260,6.273300,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.912854,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,5.000000,641.065193,7480.592129,48.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


In [19]:
# посмотрим на Square > 300
data.loc[data.Square>300]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
4262,28,9,2.0,604.705972,NaN,6.2733,17,18.0,1977,0.161532,B,B,25,5648,1,2,4,B,187717.242538
4690,2307,102,1.0,409.425181,410.639749,10.0000,4,4.0,2016,0.238617,B,B,26,3889,6,3,6,B,90470.430830
6977,11602,30,2.0,641.065193,638.163193,10.0000,20,19.0,2019,0.000078,B,B,22,6398,141,3,23,B,133529.681562


In [20]:
# гипотеза №7
# учитывая количество комнат можно предположить, что были допущены ошибки при формировании данных, заменим ошибочные данные
data.loc[4262, 'Square'] = 60.4705972
data.loc[4690, 'Square'] = 40.9425181
data.loc[6977, 'Square'] = 64.1065193

In [21]:
# гипотеза №8 
# для заполнения данных по `LifeSquare`, которые отсутствуют, примем LifeSquare = Square - KitchenSquare - Square_other_mean
data['LifeSquare'] = data['LifeSquare'].fillna(data['Square'] - data['KitchenSquare'] - Square_other_mean)

In [22]:
# посмотрим на LifeSquare < 10
data.loc[data.LifeSquare<10]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
26,4378,27,3.0,106.958871,0.641822,6.2733,17,0.0,2018,0.072158,B,B,2,629,1,0,0,A,337299.867936
95,4280,34,2.0,61.334968,4.091945,6.2733,26,1.0,2014,0.069753,B,B,53,13670,4,1,11,B,286564.441972
134,12613,6,2.0,65.496674,5.589324,6.2733,5,17.0,1977,0.243205,B,B,5,1564,0,0,0,B,180727.643541
158,16779,27,3.0,51.991064,4.896202,6.2733,22,25.0,1977,0.072158,B,B,2,629,1,0,0,A,174687.566186
160,3522,6,1.0,66.419841,3.987101,6.2733,2,1.0,1977,0.243205,B,B,5,1564,0,0,0,B,193361.229815
270,15215,17,1.0,46.692974,5.941401,6.2733,7,1.0,2015,0.000000,B,B,18,3374,5,1,2,B,212290.684086
279,8727,1,2.0,79.521221,4.634049,6.2733,5,1.0,2017,0.007122,B,B,1,264,0,0,1,B,232726.628137
363,15751,9,1.0,44.171793,1.251846,6.2733,11,18.0,2017,0.161532,B,B,25,5648,1,2,4,B,134910.715588
475,6333,27,2.0,64.049695,4.008230,6.2733,5,5.0,2017,0.017647,B,B,2,469,0,0,0,B,189477.560210
477,8849,45,1.0,63.419553,2.286357,6.2733,3,1.0,1977,0.195781,B,B,23,5212,6,3,2,B,325600.765809


In [23]:
# гипотеза №9 
# понимая, что значения по данному условию принимают аномальные значения, при чем при условии 10 < `LifeSquare` < 15,
# данные аномалии не прослеживаются,
# для заполнения данных по `LifeSquare` < 10 также примем условие LifeSquare = Square - KitchenSquare - Square_other_mean
data.loc[data.LifeSquare<10, 'LifeSquare'] = data['Square'] - data['KitchenSquare'] - Square_other_mean

In [24]:
# гипотеза №10
# посмотрим на LifeSquare < 10, чтобы определить наблюдения, которые не подошли под условие гипотезы №9
# примем для них среднее значение LifeSquare
data.loc[data.LifeSquare<10, 'LifeSquare'] = data.LifeSquare.mean()

In [25]:
# посмотрим на LifeSquare > 150
data.loc[data.LifeSquare>150]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
590,14990,23,2.0,48.449873,263.542020,5.0000,6,5.0,1972,0.075779,B,B,6,1437,3,0,2,B,141780.231857
1641,11055,151,1.0,174.649522,174.837082,15.0000,10,12.0,1958,0.236108,B,B,16,3893,27,3,10,B,232766.187698
1812,5602,93,3.0,169.385184,152.821867,6.2733,2,10.0,1977,0.004949,B,B,0,186,0,0,0,B,190949.293998
1981,7917,27,1.0,212.932361,211.231125,6.2733,2,3.0,2008,0.211401,B,B,9,1892,0,0,1,B,302211.260887
1982,5548,86,5.0,275.645284,233.949309,26.0000,12,37.0,2011,0.161976,B,A,31,7010,5,3,7,B,455264.882666
2603,5621,23,3.0,163.495333,161.504222,12.0000,5,3.0,1977,0.014073,B,B,2,475,0,0,0,B,207007.956663
3651,5300,30,3.0,169.509941,170.713651,6.2733,2,2.0,2018,0.000078,B,B,22,6398,141,3,23,B,200800.673767
4047,15362,23,1.0,163.286965,161.155275,6.2733,4,3.0,1977,0.014073,B,B,2,475,0,0,0,B,228572.520347
4328,16550,27,3.0,81.694417,7480.592129,6.2733,9,17.0,2016,0.017647,B,B,2,469,0,0,0,B,217357.492366
4690,2307,102,1.0,40.942518,410.639749,10.0000,4,4.0,2016,0.238617,B,B,26,3889,6,3,6,B,90470.430830


In [26]:
# гипотеза №11 
# для заполнения данных по `LifeSquare` > 150 также примем условие LifeSquare = Square - KitchenSquare - Square_other_mean
data.loc[data.LifeSquare>150, 'LifeSquare'] = data['Square'] - data['KitchenSquare'] - Square_other_mean

In [27]:
# посмотрим еще раз на LifeSquare > 150, можно определить, что анамольных значений не прослеживается
data.loc[data.LifeSquare>150]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1812,5602,93,3.0,169.385184,152.821867,6.2733,2,10.0,1977,0.004949,B,B,0,186,0,0,0,B,190949.293998
1981,7917,27,1.0,212.932361,196.369044,6.2733,2,3.0,2008,0.211401,B,B,9,1892,0,0,1,B,302211.260887
1982,5548,86,5.0,275.645284,239.355267,26.0000,12,37.0,2011,0.161976,B,A,31,7010,5,3,7,B,455264.882666
3651,5300,30,3.0,169.509941,152.946624,6.2733,2,2.0,2018,0.000078,B,B,22,6398,141,3,23,B,200800.673767
6201,10678,17,4.0,170.016872,153.453555,6.2733,12,0.0,1977,0.093443,B,B,23,4635,5,2,4,B,531049.721966
7084,5376,93,2.0,170.379494,153.816178,6.2733,3,2.0,1977,0.004949,B,B,0,186,0,0,0,B,194582.912756
8131,4991,93,3.0,167.767027,151.203711,6.2733,1,1.0,1977,0.004949,B,B,0,186,0,0,0,B,211180.794219
8646,11764,93,3.0,169.117203,152.553887,6.2733,5,3.0,1977,0.004949,B,B,0,186,0,0,0,B,211325.577641
9910,16568,27,4.0,200.334539,165.044523,25.0000,1,2.0,2013,0.041116,B,B,53,14892,4,1,4,B,528560.506016


In [28]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.887500,56.215413,38.377993,7.486097,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.811486,19.082339,17.820167,2.519404,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,17.414181,10.248270,2.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.780320,24.434019,6.273300,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.529412,34.058480,6.273300,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.894295,47.036284,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,5.000000,275.645284,239.355267,48.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


Будем считать, что признаки `KitchenSquare`- площадь кухни, `LifeSquare` - жилая площадь квартиры и `Square` - площадь квартиры мы исследовали.

Далее исследуем признакы `Floor`- этаж, `HouseFloor` - всего этажей у дома.

In [29]:
data.loc[data.Floor>data.HouseFloor]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
7,11993,74,2.0,80.312926,63.749609,6.2733,14,0.0,1977,7.577876e-02,B,B,6,1437,3,0,2,B,221244.156664
14,10953,27,1.0,53.769097,52.408027,6.2733,5,4.0,1977,7.215758e-02,B,B,2,629,1,0,0,A,140647.565937
16,2119,27,2.0,49.360648,31.993964,5.0000,6,5.0,1983,5.181543e-02,B,B,5,1227,0,0,0,B,117000.381287
21,11935,27,2.0,64.711835,48.148518,6.2733,15,1.0,1977,2.114012e-01,B,B,9,1892,0,0,1,B,127200.026511
23,6641,54,3.0,118.907612,102.344295,6.2733,2,0.0,1977,6.076152e-03,B,B,30,5285,0,6,6,B,571069.052600
26,4378,27,3.0,106.958871,90.395555,6.2733,17,0.0,2018,7.215758e-02,B,B,2,629,1,0,0,A,337299.867936
35,6486,200,3.0,85.280389,58.447967,9.0000,6,5.0,1960,0.000000e+00,B,B,33,7425,1,2,5,B,402871.916317
39,9371,23,2.0,60.503248,43.939932,6.2733,16,0.0,1977,3.465608e-02,B,B,0,168,0,0,0,B,229778.057902
44,10521,38,3.0,104.211396,106.340403,6.2733,20,0.0,2017,6.075305e-02,B,B,15,2787,2,0,7,B,435462.048070
51,10103,94,1.0,35.280894,23.354176,6.0000,11,9.0,1971,2.827977e-01,B,B,33,8667,2,0,6,B,148862.210174


In [30]:
# гипотеза №12
# т.к. этаж квартиры не может быть выше этажности дома, для такой выборки установим этажность дома = этаж квартиры
data.loc[data.Floor>data.HouseFloor, 'HouseFloor'] = data['Floor']

Будем считать, что признаки `Floor`- этаж, `HouseFloor` - всего этажей у дома мы исследовали.

Далее исследуем признак `HouseYear`- год постройки.

In [31]:
data.loc[data.HouseYear>2020]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,20.239714,9.0,9,12.0,20052011,0.136330,B,B,30,6141,10,3,6,B,254084.534396
4189,11607,147,2.0,44.791836,28.360393,5.0,4,9.0,4968,0.319809,B,B,25,4756,16,5,8,B,243028.603096


In [32]:
# гипотеза №13
# заменим HouseYear =20052011 - средним 2008, а 4968 - 1968, т.к. скорее всего допущена ошибка.
data.loc[1497, 'HouseYear'] = 2008
data.loc[4189, 'HouseYear'] = 1968

In [33]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.887500,56.215413,38.377993,7.486097,8.526700,13.435700,1984.86600,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.811486,19.082339,17.820167,2.519404,5.241148,6.094012,18.41187,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,17.414181,10.248270,2.000000,1.000000,1.000000,1910.00000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.780320,24.434019,6.273300,4.000000,9.000000,1974.00000,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.529412,34.058480,6.273300,7.000000,14.000000,1977.00000,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.894295,47.036284,9.000000,12.000000,17.000000,2001.00000,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,5.000000,275.645284,239.355267,48.000000,42.000000,117.000000,2020.00000,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


Будем считать, что признак `HouseYear`- год постройки. мы исследовали.

Далее исследуем признаки `Ecology_2`, `Ecology_3`, `Shops_2`, которые переведем в dummy-переменные.

Признаки `Ecology_1`, `Social_1`, `Social_2`, `Social_3`, `Helthcare_2`, `Shops_1` оставим без изменений.

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(7), int64(9), object(3)
memory usage: 1.4+ MB


In [35]:
data=pd.get_dummies(data)
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.0000,7,9.0,1969,0.089040,...,5,0,11,184966.930730,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.0000,7,9.0,1978,0.000070,...,1,1,16,300009.450063,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,6.2733,8,12.0,1968,0.049637,...,0,1,3,220925.908524,0,1,0,1,0,1
3,5809,58,2.0,53.352981,52.731512,9.0000,8,17.0,1977,0.437885,...,3,0,5,175616.227217,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.0000,11,12.0,1976,0.012339,...,1,2,4,150226.531644,0,1,0,1,0,1


Исследуем признак `DistrictId` и предположим гипотезу №14, создав синтетическую переменную стоимости кв.метра квартиры в определенном районе.

In [36]:
# гипотеза №14
data['price_per_sq']=data['Price']/data['Square']
#data.groupby(['DistrictId', 'Rooms'])['price_per_sq'].mean()
stats=data.groupby(['DistrictId','Rooms'],as_index=False)[['price_per_sq']].mean().rename(columns={'price_per_sq':'Mean_price_per_sq'})
stats.head()

,DistrictId,Rooms,Mean_price_per_sq
0,0,1.0,3549.874104
1,0,2.0,3631.502943
2,0,3.0,3774.993359
3,1,1.0,3398.879056
4,1,2.0,3164.233278


In [37]:
def join_stats(df,stats,source_df,mode='train'):
    df=pd.merge(df,stats,on=['DistrictId','Rooms'],how='left')
    if mode=='test':
        df['Mean_price_per_sq']=df['Mean_price_per_sq'].fillna(source_df['Mean_price_per_sq'].mean())
    return df

In [38]:
# присоединяем синтетическую переменную к нашему датафрейму
data=join_stats(data,stats,data)

In [39]:
# удаляем вспомогательный признак 'price_per_sq'
data = data.drop('price_per_sq', axis=1)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 23 columns):
Id                   10000 non-null int64
DistrictId           10000 non-null int64
Rooms                10000 non-null float64
Square               10000 non-null float64
LifeSquare           10000 non-null float64
KitchenSquare        10000 non-null float64
Floor                10000 non-null int64
HouseFloor           10000 non-null float64
HouseYear            10000 non-null int64
Ecology_1            10000 non-null float64
Social_1             10000 non-null int64
Social_2             10000 non-null int64
Social_3             10000 non-null int64
Helthcare_2          10000 non-null int64
Shops_1              10000 non-null int64
Price                10000 non-null float64
Ecology_2_A          10000 non-null uint8
Ecology_2_B          10000 non-null uint8
Ecology_3_A          10000 non-null uint8
Ecology_3_B          10000 non-null uint8
Shops_2_A            10000 non-null ui

In [41]:
data.drop('Id', axis=1, inplace=True)

In [42]:
data.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B,Mean_price_per_sq
0,35,2.0,47.981561,29.442751,6.0000,7,9.0,1969,0.089040,33,...,0,11,184966.930730,0,1,0,1,0,1,4320.150470
1,41,3.0,65.683640,40.049543,8.0000,7,9.0,1978,0.000070,46,...,1,16,300009.450063,0,1,0,1,0,1,4143.130929
2,53,2.0,44.947953,29.197612,6.2733,8,12.0,1968,0.049637,34,...,1,3,220925.908524,0,1,0,1,0,1,4741.247035
3,58,2.0,53.352981,52.731512,9.0000,8,17.0,1977,0.437885,23,...,0,5,175616.227217,0,1,0,1,0,1,2962.604453
4,99,1.0,39.649192,23.776169,7.0000,11,12.0,1976,0.012339,35,...,2,4,150226.531644,0,1,0,1,0,1,4209.038197


In [43]:
X = data.drop('Price', axis=1) #признаки по которым будем строить модели
y = data['Price'] # целевая переменная

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 121)

Обучим модель 2 способами с нормализацией и без нормализации

In [45]:
# для способа №2 нормализуем значения наблюдений
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train_norm = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

X_test_norm = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [46]:
#вызовем модели LinearRegression, Ridge и RandomForest
model1 = LR()
model2 = Ridge()
model3 = RF()

model1_norm = LR()
model2_norm = Ridge()
model3_norm = RF()

In [47]:
# обучим модели для способа №1
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)

# обучим модели для способа №2
model1_norm.fit(X_train_norm, y_train)
model2_norm.fit(X_train_norm, y_train)
model3_norm.fit(X_train_norm, y_train)

D:\programs\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\programs\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [48]:
# Предскажим целевую переменную по каждой из моделей для способа №1
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)
y_pred3 = model3.predict(X_test)

# Предскажим целевую переменную по каждой из моделей для способа №2
y_pred1_norm = model1_norm.predict(X_test_norm)
y_pred2_norm = model2_norm.predict(X_test_norm)
y_pred3_norm = model3_norm.predict(X_test_norm)

In [49]:
# опеределим коэффициент детерминации(r2) и среднюю квадратичную ошибку(MSE)
print('Линейная регрессия для способа №1', 'r2:', r2(y_test, y_pred1), 'MSE:', MSE(y_test, y_pred1)) 
print('Линейная регрессия для способа №2', 'r2:', r2(y_test, y_pred1_norm), 'MSE:', MSE(y_test, y_pred1_norm)) 

Линейная регрессия для способа №1 r2: 0.7059392917557463 MSE: 2552357240.7114644
Линейная регрессия для способа №2 r2: 0.705939291755742 MSE: 2552357240.711501


In [50]:
print('Ridge регрессия для способа №1', 'r2:', r2(y_test, y_pred2), 'MSE:', MSE(y_test, y_pred2)) 
print('Ridge регрессия для способа №2', 'r2:', r2(y_test, y_pred2_norm), 'MSE:', MSE(y_test, y_pred2_norm)) 

Ridge регрессия для способа №1 r2: 0.7059380269354172 MSE: 2552368218.966072
Ridge регрессия для способа №2 r2: 0.7047005697330119 MSE: 2563108969.981356


In [51]:
print('Случайный лес для способа №1', 'r2:', r2(y_test, y_pred3), 'MSE:', MSE(y_test, y_pred3)) 
print('Случайный лес для способа №2', 'r2:', r2(y_test, y_pred3_norm), 'MSE:', MSE(y_test, y_pred3_norm)) 

Случайный лес для способа №1 r2: 0.7560382069959041 MSE: 2117513939.7193627
Случайный лес для способа №2 r2: 0.7651716843066341 MSE: 2038238142.9422147


### Загрузим test

In [52]:
db_test = pd.read_csv('test.csv')
data_test = db_test.copy()
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [53]:
#иключим из test'a признак Healthcare_1
data_test = data_test.drop('Healthcare_1', axis=1)

In [54]:
#исследуем также, как и в тренировочной выборке признак Rooms
data_test.loc[data_test.Rooms>5]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2071,10793,23,6.0,110.750226,NaN,0.0,2,2.0,2015,0.014073,B,B,2,475,0,0,0,B
3217,4058,27,6.0,223.453689,104.113552,16.0,2,2.0,2017,0.041116,B,B,53,14892,4,1,4,B
3398,1435,111,17.0,52.866107,32.528342,8.0,15,17.0,1987,0.093443,B,B,23,4635,5,2,4,B


In [55]:
#приведем кол-во комнат > 5 к кол-ву комнат в зависимости от Square
data_test.loc[2071, 'Rooms']=3
data_test.loc[3217, 'Rooms']=5
data_test.loc[3398, 'Rooms']=2

In [56]:
data_test.loc[data_test.Rooms<1]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2406,3343,58,0.0,116.824201,113.692424,0.0,3,3.0,1977,0.437885,B,B,23,5735,3,0,5,B
2524,10729,27,0.0,76.345154,42.820796,12.0,14,0.0,1977,0.017647,B,B,2,469,0,0,0,B


In [57]:
# также считаем квартиры с Rooms=0, квартирами Rooms=1
data_test.loc[data_test.Rooms<1, 'Rooms']=1

Признак `Rooms` мы исследовали.

Посмотрим признакы `KitchenSquare`- площадь кухни, `LifeSquare` - жилая площадь квартиры и `Square` - площадь квартиры.

In [58]:
data_test.loc[data_test.KitchenSquare<2]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,0,2,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,0,0,A
5,7633,53,1.0,40.675627,NaN,1.0,21,21.0,1977,0.049637,B,B,34,7759,0,1,3,B
6,13329,23,2.0,68.099538,64.843025,1.0,2,17.0,1977,0.075779,B,B,6,1437,3,0,2,B
11,3310,6,3.0,84.625350,NaN,1.0,6,17.0,1977,0.243205,B,B,5,1564,0,0,0,B
14,5214,27,1.0,37.555197,NaN,1.0,5,1.0,1977,0.211401,B,B,9,1892,0,0,1,B
18,3495,27,3.0,82.652818,5.690155,1.0,4,17.0,2015,0.072158,B,B,2,629,1,0,0,A
21,16688,1,1.0,44.878767,NaN,1.0,1,9.0,1977,0.007122,B,B,1,264,0,0,1,B
22,12694,27,3.0,83.670032,46.358356,0.0,8,0.0,2015,0.072158,B,B,2,629,1,0,0,A
24,8968,27,2.0,69.849239,NaN,0.0,3,0.0,1977,0.011654,B,B,4,915,0,0,0,B


In [59]:
# Площадь кухни меньше 2 заполняем средней площадью кухни из `data`.
data_test.loc[data_test.KitchenSquare<2, 'KitchenSquare'] = data.KitchenSquare.mean()
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.906600,56.449500,36.158810,8.023357,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.807469,19.092787,17.825287,9.338849,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,1.000000,1.378543,0.333490,2.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,7.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,7.486097,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,5.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


In [60]:
data_test.loc[data_test.KitchenSquare>50]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
40,5428,27,2.0,62.326044,NaN,61.0,12,17.0,1977,0.072158,B,B,2,629,1,0,0,A
1456,5260,73,3.0,69.358242,51.247581,65.0,6,6.0,1931,0.042032,B,B,37,6856,84,2,5,B
1777,3341,62,3.0,112.114019,112.247841,112.0,3,3.0,2017,0.072158,B,B,2,629,1,0,0,A
1964,14594,11,2.0,42.795304,24.223770,620.0,11,14.0,1972,0.038693,B,B,28,6533,1,2,5,B
3816,12612,27,2.0,60.988496,33.646726,60.0,5,17.0,2013,0.072158,B,B,2,629,1,0,0,A
4281,8015,27,1.0,66.099096,33.639611,62.0,3,7.0,2016,0.014058,B,B,1,290,0,0,0,B
4405,5199,27,2.0,59.054990,61.647531,57.0,13,12.0,2016,0.211401,B,B,9,1892,0,0,1,B
4555,12640,6,2.0,54.629142,31.486308,97.0,4,17.0,2015,0.243205,B,B,5,1564,0,0,0,B


In [61]:
# посчитаем площадь кухни больше 50 так же - аномальной и заполним средним значением
data_test.loc[data_test.KitchenSquare>50, 'KitchenSquare'] = data.KitchenSquare.mean()
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.906600,56.449500,36.158810,7.808535,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.807469,19.092787,17.825287,2.385394,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,1.000000,1.378543,0.333490,2.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,7.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,7.486097,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,5.000000,223.453689,303.071094,44.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


In [62]:
# посмотрим на Square < 20
data_test.loc[(data_test.Square<20)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2,5480,190,1.0,13.597819,15.948246,12.000000,2,5.0,1909,0.000000,B,B,30,7538,87,5,5,B
66,9011,53,1.0,1.378543,1.353573,7.486097,1,1.0,1977,0.049637,B,B,34,7759,0,1,3,B
557,3458,101,1.0,19.043894,13.941145,3.000000,4,5.0,1970,0.225825,A,B,41,6558,7,3,2,B
608,16401,30,1.0,2.645046,4.338755,7.486097,2,1.0,1977,0.000078,B,B,22,6398,141,3,23,B
837,2138,27,1.0,5.647458,1.501582,7.486097,1,1.0,1977,0.017647,B,B,2,469,0,0,0,B
1165,10120,6,1.0,5.100672,3.861780,7.486097,3,1.0,1977,0.243205,B,B,5,1564,0,0,0,B
1195,3969,101,1.0,18.149267,9.698766,3.000000,4,5.0,1967,0.225825,A,B,41,6558,7,3,2,B
1441,9884,29,1.0,17.273699,16.452172,2.000000,3,5.0,1962,0.069660,B,B,31,6119,4,1,2,B
2744,7533,66,1.0,16.319015,11.118981,7.486097,4,5.0,1965,0.111627,B,B,50,12238,8,2,3,B
2895,6894,84,1.0,19.166821,14.865172,7.486097,6,5.0,1965,0.149666,B,B,22,4789,2,4,1,B


In [63]:
# скорректируем данные [Square < 20] = LifeSquare + KitchenSquare + Square_other_mean
# для этого исключим из выборки пустые значения LifeSquare
data_test.loc[(data_test.Square<20) & (data_test.LifeSquare>0), 'Square'] = data_test['LifeSquare'] + data_test['KitchenSquare'] + Square_other_mean

In [64]:
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.906600,56.506433,36.158810,7.808535,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.807469,18.993897,17.825287,2.385394,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,1.000000,19.129687,0.333490,2.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.912403,23.092026,7.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.924953,32.925087,7.486097,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.289167,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,5.000000,223.453689,303.071094,44.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


In [65]:
# посмотрим на Square > 300
data_test.loc[data_test.Square>300]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2


In [66]:
# для заполнения данных по `LifeSquare`, которые отсутствуют, примем LifeSquare = Square - KitchenSquare - Square_other_mean
data_test['LifeSquare'] = data_test['LifeSquare'].fillna(data_test['Square'] - data_test['KitchenSquare'] - Square_other_mean)

In [67]:
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.906600,56.506433,37.475836,7.808535,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.807469,18.993897,18.240916,2.385394,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,1.000000,19.129687,0.333490,2.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.912403,23.710857,7.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.924953,33.764362,7.486097,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.289167,46.741927,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,5.000000,223.453689,303.071094,44.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


In [68]:
# посмотрим на LifeSquare < 10
data_test.loc[data_test.LifeSquare<10]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
18,3495,27,3.0,82.652818,5.690155,7.486097,4,17.0,2015,0.072158,B,B,2,629,1,0,0,A
66,9011,53,1.0,19.129687,1.353573,7.486097,1,1.0,1977,0.049637,B,B,34,7759,0,1,3,B
71,9205,120,2.0,85.911827,4.188395,7.486097,5,1.0,2016,0.341072,B,B,27,5664,48,3,11,B
77,7034,45,2.0,59.927502,3.798341,7.486097,23,1.0,2017,0.195781,B,B,23,5212,6,3,2,B
78,6553,34,5.0,122.375273,0.333490,7.486097,9,0.0,1977,0.069753,B,B,53,13670,4,1,11,B
153,1105,9,3.0,105.351051,5.589869,7.486097,11,1.0,1977,0.161532,B,B,25,5648,1,2,4,B
237,5857,26,1.0,56.688460,2.132381,7.486097,8,20.0,2013,0.000000,B,B,36,6714,2,0,2,B
259,13706,23,3.0,81.003455,2.366869,7.486097,5,5.0,2017,0.034656,B,B,0,168,0,0,0,B
325,940,27,1.0,48.674779,2.912992,7.486097,7,17.0,2016,0.072158,B,B,2,629,1,0,0,A
326,14959,1,2.0,64.112441,1.181139,7.486097,13,17.0,1977,0.007122,B,B,1,264,0,0,1,B


In [69]:
# понимая, что значения по данному условию принимают аномальные значения,
# для заполнения данных по `LifeSquare` < 10 также примем условие LifeSquare = Square - KitchenSquare - Square_other_mean
data_test.loc[data_test.LifeSquare<10, 'LifeSquare'] = data_test['Square'] - data_test['KitchenSquare'] - Square_other_mean

In [70]:
data_test.loc[data_test.LifeSquare<10]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
66,9011,53,1.0,19.129687,1.353573,7.486097,1,1.0,1977,0.049637,B,B,34,7759,0,1,3,B
608,16401,30,1.0,22.114869,4.338755,7.486097,2,1.0,1977,0.000078,B,B,22,6398,141,3,23,B
837,2138,27,1.0,19.277696,1.501582,7.486097,1,1.0,1977,0.017647,B,B,2,469,0,0,0,B
1165,10120,6,1.0,21.637894,3.861780,7.486097,3,1.0,1977,0.243205,B,B,5,1564,0,0,0,B
1195,3969,101,1.0,22.988783,9.698766,3.000000,4,5.0,1967,0.225825,A,B,41,6558,7,3,2,B
1436,12833,27,1.0,27.658462,9.882348,7.486097,10,19.0,1977,0.211401,B,B,9,1892,0,0,1,B
3697,15435,21,1.0,27.271481,9.495367,7.486097,6,5.0,1965,0.194489,B,B,47,8004,3,3,5,B
4234,15846,34,1.0,26.719723,8.943610,7.486097,4,2.0,2015,0.069753,B,B,53,13670,4,1,11,B
4490,1165,27,1.0,19.675233,1.899119,7.486097,2,17.0,1977,0.011654,B,B,4,915,0,0,0,B
4540,7855,6,1.0,20.744200,2.968086,7.486097,3,1.0,2018,0.243205,B,B,5,1564,0,0,0,B


In [71]:
# посмотрим на LifeSquare < 10, чтобы определить наблюдения, которые не подошли под условие
# примем для них среднее значение LifeSquare
data_test.loc[data_test.LifeSquare<10, 'LifeSquare'] = data.LifeSquare.mean()

In [72]:
# посмотрим на LifeSquare > 150
data_test.loc[data_test.LifeSquare>150]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
410,11533,94,2.0,48.713443,303.071094,6.000000,5,12.0,1974,0.521867,B,B,25,6149,0,0,0,B
2557,16053,17,4.0,168.729035,169.901701,7.486097,4,0.0,2013,0.093443,B,B,23,4635,5,2,4,B


In [73]:
#скорректируем данные для LifeSquare > 150
data_test.loc[data_test.LifeSquare>150, 'LifeSquare'] = data_test['Square'] - data_test['KitchenSquare'] - Square_other_mean

In [74]:
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.906600,56.506433,38.401509,7.808535,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.807469,18.993897,17.409517,2.385394,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,1.000000,19.129687,10.248887,2.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.912403,24.399513,7.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.924953,34.297545,7.486097,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.289167,47.148329,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,5.000000,223.453689,150.952921,44.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


Далее исследуем признакы `Floor`- этаж, `HouseFloor` - всего этажей у дома.

In [75]:
data_test.loc[data_test.Floor>data_test.HouseFloor]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
1,15856,74,2.0,69.263183,51.487070,7.486097,6,1.0,1977,0.075779,B,B,6,1437,3,0,2,B
10,15975,61,3.0,64.543658,42.651541,7.000000,10,9.0,1981,0.300323,B,B,52,10311,6,1,9,B
14,5214,27,1.0,37.555197,19.779083,7.486097,5,1.0,1977,0.211401,B,B,9,1892,0,0,1,B
22,12694,27,3.0,83.670032,46.358356,7.486097,8,0.0,2015,0.072158,B,B,2,629,1,0,0,A
24,8968,27,2.0,69.849239,52.073126,7.486097,3,0.0,1977,0.011654,B,B,4,915,0,0,0,B
27,11160,58,1.0,48.610661,48.752502,7.486097,4,3.0,1977,0.437885,B,B,23,5735,3,0,5,B
30,2982,6,2.0,63.460684,45.684571,7.486097,13,0.0,1977,0.243205,B,B,5,1564,0,0,0,B
32,2449,1,1.0,66.426585,48.650471,7.486097,14,0.0,1977,0.007122,B,B,1,264,0,0,1,B
36,9348,205,1.0,37.480811,16.851795,8.000000,6,5.0,1933,0.169091,B,B,19,3856,10,2,5,B
41,12749,21,2.0,49.011976,27.980195,5.000000,6,5.0,1969,0.194489,B,B,47,8004,3,3,5,B


In [76]:
# т.к. этаж квартиры не может быть выше этажности дома, для такой выборки установим этажность дома = этаж квартиры
data_test.loc[data_test.Floor>data_test.HouseFloor, 'HouseFloor'] = data_test['Floor']

Далее исследуем признак `HouseYear`- год постройки.

In [77]:
data_test.loc[data_test.HouseYear>2020]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2


Далее исследуем признаки `Ecology_2`, `Ecology_3`, `Shops_2`, которые переведем в dummy-переменные.

Признаки `Ecology_1`, `Social_1`, `Social_2`, `Social_3`, `Helthcare_2`, `Shops_1` оставим без изменений.

In [78]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(6), int64(9), object(3)
memory usage: 703.2+ KB


In [79]:
data_test=pd.get_dummies(data_test)
data_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,725,58,2.0,49.882643,33.432782,6.000000,6,14.0,1972,0.310199,...,2748,1,0,0,0,1,0,1,0,1
1,15856,74,2.0,69.263183,51.487070,7.486097,6,6.0,1977,0.075779,...,1437,3,0,2,0,1,0,1,0,1
2,5480,190,1.0,38.238263,15.948246,12.000000,2,5.0,1909,0.000000,...,7538,87,5,5,0,1,0,1,0,1
3,15664,47,2.0,73.046609,51.940842,9.000000,22,22.0,2007,0.101872,...,4583,3,3,3,0,1,0,1,0,1
4,14275,27,1.0,47.527111,43.387569,7.486097,17,17.0,2017,0.072158,...,629,1,0,0,0,1,0,1,1,0


In [80]:
# присоединяем синтетическую переменную `Mean_price_per_sq` к нашему тестовому датафрейму
data_test=join_stats(data_test,stats,data,mode='test')
data_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B,Mean_price_per_sq
0,725,58,2.0,49.882643,33.432782,6.000000,6,14.0,1972,0.310199,...,1,0,0,0,1,0,1,0,1,2962.604453
1,15856,74,2.0,69.263183,51.487070,7.486097,6,6.0,1977,0.075779,...,3,0,2,0,1,0,1,0,1,3289.878362
2,5480,190,1.0,38.238263,15.948246,12.000000,2,5.0,1909,0.000000,...,87,5,5,0,1,0,1,0,1,3897.049770
3,15664,47,2.0,73.046609,51.940842,9.000000,22,22.0,2007,0.101872,...,3,3,3,0,1,0,1,0,1,3841.195532
4,14275,27,1.0,47.527111,43.387569,7.486097,17,17.0,2017,0.072158,...,1,0,0,0,1,0,1,1,0,2896.603397


In [81]:
test = data_test.drop('Id', axis=1)

Предскажем целевую переменную также 2 способами с нормализацией и без нормализации

In [82]:
# нормализуем значения наблюдений датафрейма test

test_norm = pd.DataFrame(scaler.transform(test), columns=test.columns)

In [87]:
data_test['Price']=model3.predict(test)
data_test['Price_norm']=model3_norm.predict(test_norm)

In [88]:
data_test[['Id', 'Price']].to_csv('AKashipov_predictions(для зачета).csv',index=False)
data_test[['Id', 'Price_norm']].rename(columns={'Price_norm':'Price'}).to_csv('AKashipov_predictions(экспериментально, с учетом нормализации).csv',index=False)